In [1]:
import itertools
import os

import numpy as np
import pandas as pd

In [2]:
fuzzers = ['zest', 'bedivfuzz', 'bedivfuzz-split']
subjects = ['ant', 'bcel', 'chocopy', 'closure', 'maven', 'nashorn', 'pngj', 'rhino', 'tomcat']
num_trials = 30
timeout = 180

In [3]:
def coverage_trial_df(experiment_name: str, fuzzer: str, subject: str, trial: int):
    df = pd.read_csv(
        os.path.join('results', experiment_name, fuzzer, subject, f'trial-{trial}', 'plot_data'),
        skipinitialspace=True
    )

    if fuzzer == 'bedivfuzz-split':
        zest_plot_data = pd.read_csv(
            os.path.join('results', experiment_name , fuzzer, subject, f'trial-{trial}', 'zest-results', 'plot_data'),
            skipinitialspace=True
        )

        df = pd.concat([zest_plot_data, df], ignore_index=True)

    # one datapoint per minute
    df = df.loc[np.linspace(0, len(df)-1, timeout, endpoint=True, dtype=np.int64)]
    df['time'] = range(1, timeout+1)
    df['trial'] = trial
    df['fuzzer'] = fuzzer
    df['subject'] = subject
    df['validity_rate'] = df['valid_inputs'] / (df['valid_inputs'] + df['invalid_inputs'])

    if 'unique_valid_paths' in df.columns:
        return df[[
            'fuzzer', 'subject', 'trial', 'time', 
            'valid_inputs', 'invalid_inputs', 'validity_rate', 'unique_paths', 'unique_valid_paths',
            'num_coverage_probes', 'num_semantic_probes', 'b0', 'b1', 'b2'
        ]]

    else:
        return df[[
                'fuzzer', 'subject', 'trial', 'time', 
                'valid_inputs', 'invalid_inputs', 'validity_rate', 'unique_paths',
                'num_coverage_probes', 'num_semantic_probes', 'b0', 'b1', 'b2'
            ]]

In [6]:
def coverage_results_to_csv(experiment_name):
    dfs = []
    for f, s, t in itertools.product(fuzzers, subjects, range(1, num_trials+1)):
        dfs.append(coverage_trial_df(experiment_name, fuzzer=f, subject=s, trial=t))
    trials = pd.concat(dfs)

    trials.to_csv(
        os.path.join('results', experiment_name, 'coverage-data.csv'),
        index=False
    )
    return trials

In [7]:
thesis_default = coverage_results_to_csv('eval-bedivfuzz-split-thesis-default')
metrics_default = coverage_results_to_csv('eval-bedivfuzz-metrics-default')

thesis_semantic = coverage_results_to_csv('eval-bedivfuzz-split-thesis-semantic')
metrics_semantic = coverage_results_to_csv('eval-bedivfuzz-metrics-semantic')

In [8]:
crash_to_id = {}
def deduplicate_crash(exception_class, stack_trace):
    key = (exception_class, '-'.join(str(stack_trace).split('-')[:3]))
    if str(exception_class) == 'nan':
        return -1
    if key in crash_to_id.keys():
        return crash_to_id[key]
    else:
        crash_id = len(crash_to_id)
        crash_to_id[key] = crash_id
        return crash_id

In [70]:
def crash_trial_df(experiment_name: str, fuzzer: str, subject: str, trial: int):
    # Read failure stats
    df = pd.read_csv(
        os.path.join('results', experiment_name, fuzzer, subject, f"trial-{trial}", 'failure_info.csv'),
        skipinitialspace=True
    )

    # Map TTE to failure messages
    fuzz_log = os.path.join('results', experiment_name, fuzzer, subject, f"trial-{trial}", 'fuzz.log')
    tte_to_message = {}
    if os.path.exists(fuzz_log):
        with open(fuzz_log, 'r') as f:
            for line in f:
                if 'Found failure' not in line:
                    continue
                tokens = line.split(" ")
                tte = tokens[0]
                #crash_class = tokens[6]
                message = " ".join(tokens[7:])
                tte_to_message[tte] = message

    df['fuzzer'] = fuzzer
    df['subject'] = subject
    df['trial'] = trial
    df['tte'] = df['# ttd']
    if not df.empty: # Assignment fails if df is empty
        df['exception_class'] = df.apply(lambda row: str(row['exception_class']).split("class ")[1], axis=1)
    df['crash_id'] = df.apply(lambda row: deduplicate_crash(row['exception_class'], row['top5_stack_trace']), axis=1)
    df['location'] = df.apply(lambda row: ''.join(str(row['top5_stack_trace']).split('-')[:1]), axis=1)
    df['message'] = df.apply(lambda row: tte_to_message.get(str(row['# ttd']), 'no message'), axis=1)
    df['stack_trace'] = df['top5_stack_trace']

    return df[['fuzzer', 'subject', 'trial', 'tte', 'exception_class', 'location', 'message', 'stack_trace', 'crash_id']]

In [18]:
def crash_results_to_csv(experiment_name: str):
    trial_crashes = []
    for f, s, t in itertools.product(fuzzers, subjects, range(1, num_trials+1)):
        trial_crashes.append(crash_trial_df(experiment_name, fuzzer=f, subject=s, trial=t))
    crash_data = pd.concat(trial_crashes)
    crash_data.to_csv(
        os.path.join('results', experiment_name, 'crash-data.csv'),
        index=False
    )
    return crash_data

In [19]:
thesis_default = crash_results_to_csv('eval-bedivfuzz-split-thesis-default')
metrics_default = crash_results_to_csv('eval-bedivfuzz-metrics-default')

thesis_semantic = crash_results_to_csv('eval-bedivfuzz-split-thesis-semantic')
metrics_semantic = crash_results_to_csv('eval-bedivfuzz-metrics-semantic')

# Process ICSE22 Results

In [17]:
import shutil

In [20]:
# Reorganizes the directory structure of the original artifact
move_dirs = False
experiment = 'eval-icse22-crashes'

if move_dirs:
    base_dir = os.path.join('results', experiment, 'java-data')
    for d in os.listdir(base_dir):
        results_dir = os.path.join(base_dir, d)
        if os.path.isdir(results_dir):
            tokens = d.split('-')
            tech = tokens[0]
            if tokens[1] in ['simple', 'structure']:
                tech = f"{tokens[0]}-{tokens[1]}"
                benchmark = tokens[2]
                trial_id = f"trial-{'-'.join(tokens[3:])}"
            else:
                benchmark = tokens[1]
                trial_id = f"trial-{'-'.join(tokens[2:])}" 
    
            new_results_dir = os.path.join('results', experiment, tech)
            if not os.path.exists(new_results_dir):
                os.makedirs(new_results_dir)
    
            shutil.move(results_dir, os.path.join(new_results_dir, benchmark, trial_id))

In [4]:
fuzzers = ['bediv-simple', 'bediv-structure', 'quickcheck', 'rl', 'zest']
subjects = ['ant', 'closure', 'maven', 'nashorn', 'rhino', 'tomcat']
num_trials = 30
timeout = 60

In [5]:
def icse_coverage_trial_df(fuzzer: str, subject: str, trial: int):
    if fuzzer in ['quickcheck', 'rl']:
        plot_data = os.path.join('results', 'eval-icse22-coverage', fuzzer, subject, f'trial-{trial}-replay', 'plot_data')
    else:
        plot_data = os.path.join('results', 'eval-icse22-coverage', fuzzer, subject, f'trial-{trial}', 'plot_data')

    df = pd.read_csv(
        plot_data,
        names=[
            'unix_time', 'unique_crashes', 'total_cov', 'valid_cov', 'total_inputs', 'valid_inputs', 
            'unique_valid_paths', 'unique_valid_branch_sets', 'unique_valid_inputs', 'b0', 'b1', 'b2'],
        header=None,
        skiprows=1,
        skipinitialspace=True
    )

    # two datapoints per minute
    df = df.loc[np.linspace(0, len(df)-1, 2*timeout, endpoint=True, dtype=np.int64)]
    df['time'] = np.arange(0.5, timeout + 0.5, 0.5)
    df['trial'] = trial
    df['fuzzer'] = fuzzer
    df['subject'] = subject
    df['validity_rate'] = df['valid_inputs'] / df['total_inputs']

    return df[[
            'fuzzer', 'subject', 'trial', 'time', 
            'valid_inputs', 'total_inputs', 'validity_rate', 'unique_valid_paths', 'b0', 'b1', 'b2'
        ]]

In [6]:
dfs = []
for f, s, t in itertools.product(fuzzers, subjects, range(1, num_trials+1)):
    dfs.append(icse_coverage_trial_df(fuzzer=f, subject=s, trial=t))
    
trials = pd.concat(dfs)
trials.to_csv(
    os.path.join('results', 'eval-icse22-coverage', 'coverage-data.csv'),
    index=False
)

In [73]:
def icse_crash_trial_df(fuzzer: str, subject: str, trial: int):
    def get_num_failures(failure_directory):
        num_files = len([entry for entry in os.listdir(failure_directory) if os.path.isfile(os.path.join(failure_directory, entry))])
        assert num_files % 2 == 0, failure_directory
        return int(num_files / 2) # two files per failure
    
    def read_stack_trace(file):
        with open(file, 'r') as f:
            return '-'.join([line.strip() for line in f][1:]) # skip exception class

    def read_crash_stats(file):
        with open(file, 'r') as f:
            lines = [line.strip() for line in f]
            clazz = (lines[0].split("class "))[1]
            tte = (lines[1].split("TTD: "))[1]
            return clazz, int(int(tte) / 1000) # convert ms to s

    # Map failure_id to failure message
    failure_directory = os.path.join('results', 'eval-icse22-crashes', fuzzer, subject, f"trial-{trial}", 'failure_info')
    fuzz_log = os.path.join('results', 'eval-icse22-crashes', fuzzer, subject, f"trial-{trial}", 'fuzz.log')
    failure_id_to_message = {}

    # Return empty df if no crashes have been found
    if not os.path.exists(fuzz_log):
        return pd.DataFrame(columns=['fuzzer', 'subject', 'trial', 'tte', 'exception_class', 'location', 'message', 'stack_trace', 'crash_id'])
    
    rows = []
    with open(fuzz_log, 'r') as f:
        for line in f:
            if 'Found crash' not in line:
                continue
            tokens = line.split(" ")
            tte = int(int(tokens[0]) / 1000) # convert ms to s
            failure_id = tokens[1]
            exception_class = tokens[5]
            message = " ".join(tokens[7:]) if len(tokens) > 7 else 'no message'
            stack_trace_file = os.path.join(failure_directory, f"{failure_id}.stacktrace")
            stack_trace = read_stack_trace(stack_trace_file)

            rows.append({
                    'fuzzer': fuzzer,
                    'subject': subject,
                    'trial': trial,
                    'tte': tte,
                    'exception_class': exception_class,
                    'location': stack_trace.split('-')[0],
                    'message': message,
                    'stack_trace': stack_trace,
                    'crash_id': deduplicate_icse_crash(exception_class, stack_trace.split('-')[0])
                })

    return pd.DataFrame(rows)

In [75]:
# Note: this takes *a lot* of time (because quickcheck produces so many crashes)
# Set the following flag to true if you have some spare time...
generate_icse_crash_data = False

icse_crash_to_id = {}
def deduplicate_icse_crash(exception_class, location):
    key = (exception_class, location)
    if key in icse_crash_to_id.keys():
        return icse_crash_to_id[key]
    else:
        crash_id = len(icse_crash_to_id)
        icse_crash_to_id[key] = crash_id
        return crash_id

# Generate ICSE crash data csv
if generate_icse_crash_data:
    trial_crashes = []
    for f, s, t in itertools.product(fuzzers, subjects, range(1, num_trials+1)):
        trial_crashes.append(icse_crash_trial_df(fuzzer=f, subject=s, trial=t))
        
    crash_data = pd.concat(trial_crashes)
    crash_data.to_csv(
        os.path.join('results', 'eval-icse22-crashes', 'crash-data.csv'),
        index=False
    )

In [79]:
icse_crash_to_id

{('java.lang.RuntimeException',
  'com.google.javascript.jscomp.jarjar.com.google.common.base.Preconditions.checkArgument(Preconditions.java)'): 0,
 ('java.lang.StringIndexOutOfBoundsException',
  'java.base/java.lang.StringLatin1.charAt(StringLatin1.java:47)'): 1,
 ('java.lang.RuntimeException',
  'com.google.javascript.jscomp.InlineObjectLiterals$InliningBehavior.afterExitScope(InlineObjectLiterals.java)'): 2,
 ('java.lang.RuntimeException', ''): 3,
 ('java.lang.NullPointerException',
  'com.google.javascript.jscomp.jarjar.com.google.common.base.Preconditions.checkNotNull(Preconditions.java)'): 4,
 ('java.lang.RuntimeException',
  'com.google.javascript.jscomp.VarCheck.handleUndeclaredVariableRef(VarCheck.java)'): 5,
 ('java.lang.RuntimeException',
  'com.google.javascript.jscomp.PeepholeRemoveDeadCode.tryFoldLabel(PeepholeRemoveDeadCode.java)'): 6,
 ('java.lang.AssertionError',
  'jdk.scripting.nashorn/jdk.nashorn.internal.parser.ParserContext.pop(ParserContext.java:91)'): 7,
 ('jav